## This notebook is part of self learning Quantum Computing Algorithms

## Description: This program optimizes the stock portfolio
### With real time data + increase in assests + increase in budget

In [2]:
import numpy as np
from qiskit_finance.data_providers import RandomDataProvider
from pandas_datareader import data
import datetime
import pandas as pd

In [3]:
num_assets = 6
seed = 123
start=datetime.datetime(2021,6,1)
end=datetime.datetime(2021,6,30)
# Generate expected return and covariance matrix from (random) time-series
stocks = ['AAPL', 'NKE', 'GOOGL', 'AMZN', 'FB', 'MSFT']

In [4]:
# Import data
df = data.DataReader(stocks, 'yahoo', start=start, end=end)

In [5]:
df.head()

Attributes   Adj Close                                                    \
Symbols           AAPL         NKE        GOOGL         AMZN          FB   
Date                                                                       
2021-06-01  124.094070  134.509995  2381.179932  3218.649902  329.130005   
2021-06-02  124.872902  134.169998  2370.590088  3233.989990  329.149994   
2021-06-03  123.355186  134.169998  2347.580078  3187.010010  326.040009   
2021-06-04  125.701668  133.740005  2393.570068  3206.219971  330.350006   
2021-06-07  125.711655  133.949997  2402.300049  3198.010010  336.579987   

Attributes                   Close                                        ...  \
Symbols           MSFT        AAPL         NKE        GOOGL         AMZN  ...   
Date                                                                      ...   
2021-06-01  246.927277  124.279999  134.509995  2381.179932  3218.649902  ...   
2021-06-02  246.827469  125.059998  134.169998  2370.590088  3233.989990  ...   
2021-06-03  245.240524  123.540001  134.169998  2347.580078  3187.010010  ...   
2021-06-04  250.310791  125.889999  133.740005  2393.570068  3206.219971  ...   
2021-06-07  253.325027  125.900002  133.949997  2402.300049  3198.010010  ...   

Attributes         Open                                         Volume  \
Symbols           GOOGL         AMZN          FB        MSFT      AAPL   
Date                                                                     
2021-06-01  2374.439941  3243.500000  330.149994  251.229996  67637100   
2021-06-02  2389.149902  3223.100098  330.380005  248.130005  59278900   
2021-06-03  2345.729980  3204.229980  325.779999  245.220001  76229200   
2021-06-04  2369.270020  3212.000000  325.899994  247.759995  75169300   
2021-06-07  2389.439941  3197.330078  329.480011  249.979996  71057600   

Attributes                                                 
Symbols         NKE    GOOGL     AMZN        FB      MSFT  
Date                                                       
2021-06-01  5577900  1167700  2430000  11765900  23213300  
2021-06-02  5226100  1057900  2014500  11654300  19406700  
2021-06-03  5027400   934800  2398300  12610800  25307700  
2021-06-04  5217100  1222900  2249700  13289400  25281100  
2021-06-07  3765000  1206000  2215800  20136700  23079200  

[5 rows x 36 columns]

In [6]:
df = df['Adj Close']

In [7]:
df.head()

Symbols,AAPL,NKE,GOOGL,AMZN,FB,MSFT
Date,,,,,,
2021-06-01,124.094070,134.509995,2381.179932,3218.649902,329.130005,246.927277
2021-06-02,124.872902,134.169998,2370.590088,3233.989990,329.149994,246.827469
2021-06-03,123.355186,134.169998,2347.580078,3187.010010,326.040009,245.240524
2021-06-04,125.701668,133.740005,2393.570068,3206.219971,330.350006,250.310791
2021-06-07,125.711655,133.949997,2402.300049,3198.010010,336.579987,253.325027


In [8]:
df_data = []
for stock in stocks:
    df_data.append(df[stock].values.tolist())
print(df_data)

[[124.09407043457031, 124.8729019165039, 123.35518646240234, 125.70166778564453, 125.71165466308594, 126.5503921508789, 126.93981170654297, 125.92134094238281, 127.15947723388672, 130.2847900390625, 129.44606018066406, 129.95529174804688, 131.59283447265625, 130.26483154296875, 132.10208129882812, 133.77955627441406, 133.49998474121094, 133.21041870117188, 132.91087341308594, 134.578369140625, 136.1260528564453, 136.75511169433594], [134.50999450683594, 134.1699981689453, 134.1699981689453, 133.74000549316406, 133.9499969482422, 133.35000610351562, 131.83999633789062, 130.97999572753906, 131.94000244140625, 131.36000061035156, 130.2899932861328, 130.39999389648438, 128.9199981689453, 128.41000366210938, 130.0800018310547, 132.47999572753906, 133.10000610351562, 133.60000610351562, 154.35000610351562, 152.36000061035156, 155.9499969482422, 154.49000549316406], [2381.179931640625, 2370.590087890625, 2347.580078125, 2393.570068359375, 2402.300048828125, 2398.43994140625, 2407.93994140625,

In [3]:
length = (end - start).days
generator = np.random.default_rng(123)
data = []
for _ in stocks:
    #row sum(sum revious row value with current and so on...) + add a constant over each row
    d_f = pd.DataFrame(generator.standard_normal(length)).cumsum() + generator.integers(1, 101)
    #pick the maximum value between actual and zero... if value in dataframe is +ve will returned else returned 0
    trimmed = np.maximum(d_f[0].values, np.zeros(len(d_f[0].values)))
    trimmed_list = trimmed.tolist()
    # find index of first 0 element else return -1
    zero_idx = next((idx for idx, val in enumerate(trimmed_list) if val == 0), -1)
    if zero_idx >= 0:
        # set to 0 all values after first 0
        trimmed_list = [val if idx < zero_idx else 0 for idx, val in enumerate(trimmed_list)]
    data.append(trimmed_list)
print(data)

[[16.01087864965215, 15.643091998184266, 16.931017259473514, 17.12499167860613, 18.045222578245983, 18.622326369503234, 17.985862723132257, 18.52781494354255, 18.211219492376735, 17.888830376217772, 17.98599769488823, 16.460067288369277, 17.652233392470936, 16.981143717296828, 17.981413136956288, 18.117734260809407, 19.649767340438203, 18.989797926646382, 18.678003070176462, 19.01577219673529, 16.808301098535484, 17.636222540094224, 19.17785293478484, 20.30465972804987, 21.059429372362118, 20.913451479246895, 22.19535370630661, 23.26938432827855, 23.662005172855824], [62.63823312783908, 61.408000932348635, 62.63423022516978, 60.4621863384846, 60.09203899263228, 60.25641906230695, 61.11630024691969, 62.8779614834315, 63.87128525938331, 63.57976383328487, 64.30789139117401, 63.04629107425432, 64.47622960094303, 64.31975427611363, 63.645995126126564, 63.00693502569436, 62.945573698073986, 62.55278877550404, 64.84269872281862, 64.12451757493803, 64.157125318095, 64.18517521368064, 64.21344

## $\min\limits_{x \in \{0, 1\}^{n}} q \ x^{T}\Sigma x − \mu^{T}x \\ \text{or} \\ \max\limits_{x \in \{0, 1\}^{n}} \mu^{T}x - q \ x^{T}\Sigma x \\ \text{subject to}: 1^{T} x = B$

##### $\text{ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html} \\  x \in \{0, 1\}^{n} \ \text{denotes the vector of binary decision variables, which indicate which assets to pick} \left(x[i]=1\right) \text{and which not to pick} \left(x[i]=0\right), \\ \mu \in R^{n} \ \text{defines the expected returns for the assets}, \\ \Sigma \in R^{n \times n} \text{specifies the covariances between the assets}, \\ q \gt 0 \ \text{controls the risk appetite of the decision maker}, \\ \text{and B denotes the budget, i.e. the number of assets to be selected out of n.}$

## $\text{The equality constraint} \ 1^{T}x = B \ \text{is mapped to a penalty term} \ \left(1^{T} x − B\right)^{2}$

## $\text{Assumptions}:\\ \ \cdot \text{all assets have the same price} \left(\text{normalized to 1}\right) \\ \ \cdot \text{the full budget B has to be spent, i.e., one has to select exactly B assets.}$

In [9]:
def divide_2(val_1, val_2):
    if val_2 == 0:
        if val_1 == 0:
            return 1
        return np.nan
    return val_1 / val_2

## Calculate mean returns 
## $\mu \in R^{n} \ \text{defines the expected returns for the assets}$

In [11]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## Calculate mean covariance
##  $\Sigma \in R^{n \times n} \text{specifies the covariances between the assets}$

In [12]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
sigma = np.cov(period_returns)

## plot sigma

In [85]:
import matplotlib.pyplot as plt

In [81]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## $q \gt 0 \ \text{controls the risk appetite of the decision maker}$

In [82]:
q = 0.5 # set risk factor

## $\text{B denotes the budget, i.e. the number of assets to be selected out of n.}$

In [84]:
budget = 3 # set budget (B)

### ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html

In [86]:
penalty = num_assets # set parameter to scale the budget penalty term

## Model

In [87]:
from docplex.mp.model import Model

In [88]:
mdl = Model('portfolio model')
#x = mdl.binary_var_list(num_assets)

In [89]:
x = list()
for i in range(num_assets):
    x.append(mdl.binary_var(name="x_{0}".format(i)))
print(x)

[docplex.mp.Var(type=B,name='x_0'), docplex.mp.Var(type=B,name='x_1'), docplex.mp.Var(type=B,name='x_2'), docplex.mp.Var(type=B,name='x_3'), docplex.mp.Var(type=B,name='x_4'), docplex.mp.Var(type=B,name='x_5')]


## $\max\limits_{x \in \{0, 1\}}  \mu^{T} * x - q * x^{T} * \Sigma * x \\ \text{linear} = c^{T}x, \text{qudratic} = x^{T}Qx = \sum\limits_{i, j = 1}^{n} x_{i}^{T} \Sigma_{ij} x_{j} \\ \text{objective} = \text{linear - }\left(\text{risk_factor * qudratic}\right)$

In [90]:
linear = np.dot(mu, x) # mu^T * x
qudratic = mdl.sum([x[i]*sigma[i][j]*x[j] for i in range(num_assets) for j in range(num_assets)])
objective = linear - q * qudratic
mdl.maximize(objective)

## budget constraint:  $\sum\limits_{i =1}^{n} 1^{T}x_{i} == \text{budget}$

In [91]:
cost = mdl.sum([x[i] for i in range(num_assets)])
mdl.add_constraint(cost == budget, ctname='budget')

docplex.mp.LinearConstraint[budget](x_0+x_1+x_2+x_3+x_4+x_5,EQ,3)

## converting to Quadratic Program

## removing the constraint to create the QUBO

In [92]:
from qiskit_optimization.translators import from_docplex_mp

In [93]:
mod = from_docplex_mp(mdl)

In [94]:
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio model

Maximize
 obj: 0.002283514193 x_0 + 0.001420922407 x_1 + 0.001528562780 x_2
      + 0.001462444501 x_3 + 0.001764887813 x_4 + 0.001822253679 x_5 + [
      - 0.000516104593 x_0^2 - 0.000488960887 x_0*x_1 - 0.000606082614 x_0*x_2
      - 0.000581519803 x_0*x_3 - 0.000714472491 x_0*x_4 - 0.000720493875 x_0*x_5
      - 0.000426426820 x_1^2 - 0.000417654306 x_1*x_2 - 0.000288345775 x_1*x_3
      - 0.000417119314 x_1*x_4 - 0.000463995811 x_1*x_5 - 0.000381948380 x_2^2
      - 0.000486629535 x_2*x_3 - 0.000651244014 x_2*x_4 - 0.000622492230 x_2*x_5
      - 0.000363207901 x_3^2 - 0.000580860681 x_3*x_4 - 0.000559276903 x_3*x_5
      - 0.000530976674 x_4^2 - 0.000644671336 x_4*x_5 - 0.000408314894 x_5^2 ]/2
Subject To
 budget: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 = 3

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
End



## QAOA

In [38]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit import Aer

In [95]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

In [96]:
def print_result(result):
    selection = result.x
    res = list()
    for i in range(len(selection)):
        if selection[i]:
            res.append(stocks[i])
    value = result.fval
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))
    print('Optimal: selected stocks {}'.format(res))

    eigenstate = result.min_eigen_solver_result.eigenstate
    eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = QuadraticProgramToQubo().convert(mod).objective.evaluate(x)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [97]:
algorithm_globals.random_seed = 1234
backend = Aer.get_backend('statevector_simulator')
cobyla = COBYLA()
cobyla.set_options(maxiter=250)
quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
qaoa_mes = QAOA(optimizer=cobyla, reps=3, quantum_instance=quantum_instance)
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa_mes)
result = eigen_optimizer.solve(mod)
print_result(result)

Optimal: selection [1. 0. 0. 0. 1. 1.], value 0.0041
Optimal: selected stocks ['AAPL', 'FB', 'MSFT']

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 1 1 1 0]	-0.0033		0.0498
[0 1 1 1 0 0]	-0.0032		0.0498
[0 1 1 0 1 0]	-0.0033		0.0498
[0 1 0 1 1 0]	-0.0033		0.0498
[0 1 1 0 0 1]	-0.0034		0.0498
[0 0 1 1 0 1]	-0.0034		0.0498
[0 1 0 1 0 1]	-0.0035		0.0498
[0 0 1 0 1 1]	-0.0035		0.0498
[0 0 0 1 1 1]	-0.0035		0.0498
[0 1 0 0 1 1]	-0.0036		0.0498
[1 1 1 0 0 0]	-0.0038		0.0497
[1 0 0 1 1 0]	-0.0039		0.0497
[1 1 0 1 0 0]	-0.0038		0.0497
[1 0 1 1 0 0]	-0.0038		0.0497
[1 0 1 0 1 0]	-0.0039		0.0497
[1 1 0 0 1 0]	-0.0039		0.0497
[1 0 0 1 0 1]	-0.0040		0.0497
[1 1 0 0 0 1]	-0.0040		0.0497
[1 0 1 0 0 1]	-0.0040		0.0497
[1 0 0 0 1 1]	-0.0041		0.0497
[1 1 1 1 1 1]	9.1366		0.0003
[0 0 0 0 0 0]	9.1415		0.0002
[1 1 1 1 0 0]	1.0113		0.0001
[0 1 1 1 0 1]	1.0117		0.0001
[0 1 1 1 1 0]	1.0118		0.0001
[0 1 0 1 1 1]	1.0116

## If you have any queries on this notebook please reach to me bala.na@hcl.com